In [1]:
# Import necessary packages
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election, grid)
from gerrychain.metrics import mean_median, partisan_bias, polsby_popper, efficiency_gap
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import inspect
import pickle
import geopandas as gp
from shapely.geometry import LineString, Polygon
from shapely.ops import unary_union
import itertools
import math

In [2]:
# Import tools file
from merges import *
from plotting import *

In [3]:
precincts = gp.read_file("UtahData/new_shp.shp")

In [4]:
# The precincts file in the cell above is essentially equivalent to 
precincts_old = gp.read_file('UtahData/2018_precincts_w_votes_and_pop.shp')

# with the following changes made
precincts_old.loc[101, 'geometry'] = precincts_old.loc[101, 'geometry'][0]
precincts_old.loc[175, 'geometry'] = precincts_old.loc[175, 'geometry'][0]
# (deleting small islands)
precincts_old.loc[1887, 'US_Distric'] = 1
# (changing the district of a precinct in Summit County to reflect the fact that all of Summit County is in district 0)

In [5]:
# Get the graph data
utah = Graph.from_json("UtahData/graph.json")

In [6]:
# The graph in the cell above is essentially equivalent to 
graph_old = Graph.from_json('UtahData/2018_full_with_clean_buffer_graph.json')

# with the following change made:
graph_old.nodes[1887]['US_Distric'] = 1
# (changing the district of a precinct in Summit County to reflect the fact that all of Summit County is in district 0)

# Pre-processing

In [8]:
# Choose the graph and gdf we are working with
graph, gdf = utah.copy(), precincts.copy()

## District Issues

In [9]:
# Put precinct 1887 (population zero) into district 1.
# (I believe this is an error because several online sources list that all of Summit County is in district 1.)
graph.nodes[1887]['US_Distric'] = 1
gdf.loc[1887, 'US_Distric'] = 1

In [10]:
columns = ['SEN_REP', 'SEN_DEM', 'G_DEM', 'G_REP']
for column in columns:
    s = np.nonzero(gdf[column].isnull().values)
    for i in s[0]:
        gdf.iloc[i, gdf.columns.get_loc(column)] = 0

## Geometry Issues

In [11]:
# Delete small sliver
gdf.loc[40,'geometry'] = precincts.loc[40,'geometry'][0]

# Delete small sliver
gdf.loc[229, 'geometry'] = precincts.loc[229, 'geometry'][0]

# Delete small island
gdf.loc[543,'geometry'] = precincts.loc[543,'geometry'][0]

# Delete several small slivers
gdf.loc[636,'geometry'] = precincts.loc[636, 'geometry'][0]

In [12]:
# Find the multipolygons
multipolygons = []
for i, poly in enumerate(gdf['geometry']):
    if type(poly) != Polygon:
        multipolygons.append(i)
print(multipolygons)

[17, 19, 90, 107, 109, 112, 118, 128, 130, 131, 137, 139, 140, 141, 142, 146, 150, 152, 156, 161, 211, 235, 238, 239, 242, 248, 249, 262, 281, 315, 317, 318, 319, 321, 322, 323, 330, 331, 341, 343, 363, 395, 444, 515, 525, 532, 556, 557, 569, 599, 602, 604, 607, 610, 613, 615, 624, 629, 630, 634, 638, 646, 661, 670, 688, 719, 722, 749, 844, 957, 981, 1004, 1012, 1013, 1014, 1016, 1021, 1051, 1121, 1131, 1138, 1139, 1145, 1151, 1164, 1165, 1171, 1220, 1224, 1229, 1230, 1242, 1256, 1258, 1280, 1294, 1295, 1307, 1308, 1315, 1319, 1320, 1345, 1353, 1357, 1360, 1361, 1369, 1372, 1375, 1393, 1406, 1414, 1419, 1423, 1430, 1467, 1502, 1510, 1515, 1516, 1518, 1523, 1530, 1531, 1548, 1570, 1571, 1577, 1600, 1604, 1605, 1606, 1607, 1608, 1611, 1612, 1613, 1614, 1615, 1616, 1619, 1623, 1638, 1645, 1646, 1650, 1667, 1688, 1692, 1714, 1715, 1716, 1729, 1731, 1739, 1744, 1749, 1759, 1760, 1763, 1765, 1769, 1770, 1783, 1798, 1808, 1810, 1812, 1816, 1821, 1822, 1824, 1825, 1828, 1830, 1887, 1929, 1995,

In [13]:
# Delete very small islands

# necessary to amend 2164, 2468, 2664
# 1423, 1744, 2532, 2885, 2906, 2957 also included 

tol = 1 # square meter 
for mp in multipolygons:
    multip = gdf.iloc[mp]['geometry']
    if sum(poly.area for poly in multip[1:]) < tol:
        print(mp)
        gdf.loc[mp, 'geometry'] = gdf.loc[mp, 'geometry'][0]

1423
1744
2164
2468
2532
2664
2885
2906
2957


In [14]:
# Set some common parameters for merging
columns_to_keep = ['geometry', ('CountyID', 'first'), ('VistaID', 'sum'), ('POP100', 'sum'), 
                                 ('SEN_DEM', 'sum'), ('SEN_REP', 'sum'), ('G_DEM', 'sum'), ('G_REP', 'sum'),
                                 ('US_Distric', 'first'), ('UT_SEN','first'), ('UT_HOUSE','first')]

column_labels = ['geometry', 'CountyID', 'VistaID', 'POP100', 'SEN_DEM', 'SEN_REP', 'G_DEM',
                                 'G_REP' , 'US_Distric', 'UT_SEN', 'UT_HOUSE']

attributes_to_sum = ['POP100', 'area', 'SHAPE_Area', 'SEN_DEM', 'SEN_REP', 'G_DEM', 'G_REP']

attributes_to_join = ['VistaID', 'PrecinctID', 'SubPrecinc']

In [15]:
# These precincts have to be merged
merge_init = Merge(len(graph))
merge_init.add(set([2957, 2967]))
dissolve_init = merge_init.get_dissolve()

# Perform the merge
gdf_init = perform_dissolve_gdf(gdf, dissolve_init, columns_to_keep, column_labels)
graph_init = perform_dissolve_graph(graph, dissolve_init, attributes_to_sum, attributes_to_join)

C:\Users\Jacob\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


TypeError: Index does not support mutable operations

In [ ]:
# Delete two small islands
s = gdf_init.loc[2957, 'geometry']
i = np.argmax([s[i].area for i in range(len(s))])
gdf_init.loc[2957, 'geometry'] = gdf_init.loc[2957, 'geometry'][i]

In [ ]:
gdf_init.loc[2957, 'geometry']

# Section 1: Merging Multipolygons

All of the precincts geometries are either a shapely Polygon or a list of shapely Polygons, called a Multipolygon. A Polygon may not be simply connected (i. e. it may have holes) but it will be connected. For Multipolygons we have no such guarantee, and in fact we have reason to suspect that they are disconnected (otherwise they could be cast into a Polygon). To make sure that our Markov chain does not generate disconnected districts, we must ensure that each of the underlying precincts are connected. We can do this by finding all of the precincts which are of type Multipolygon, and then merging them with one of their neighbors. In my code, this choice is made by determining which of the neighbors in the same county and congressional district has the largest shared perimeter with a given district, and then merging that polygon. 

In [ ]:
# Choose the graph we operate on 
graph, gdf = graph_init, gdf_init

In [ ]:
# Find the multipolygons
multipolygons = []
for i, poly in enumerate(gdf['geometry']):
    if type(poly) != Polygon:
        multipolygons.append(i)
print(multipolygons)

In [ ]:
# Mark the multipolygons
f = gdf.copy()
mp = np.array([1 if i in multipolygons else 0 for i in range(len(f))])
f['mp'] = mp

# Plot the multipolygons
size, dpi =(14,7), 300
fig, ax = plt.subplots(figsize=size, dpi=dpi)
f.plot(column='mp', ax=ax, cmap='Spectral')
plt.axis("off")

# Legend: Red: Polygon, Blue: Multipolygon

In [ ]:
# time to run: approx 1 min. Gets stuck at 171 for a little, but don't worry
merge_mp = merge_multipolygons(graph, gdf, preserve_ut_house=True)
dissolve_mp = merge_mp.get_dissolve()

In [ ]:
np.nonzero(dissolve_mp == dissolve_mp[1825])[0]

In [ ]:
# Get the multipolygons along with the nodes they will be merged with
f = gdf.copy()
s = set([subpart for part in merge_mp.merges for subpart in part])
mps_and_neighbors = np.array([1 if (n in s) else 0 for n in range(len(f)) ])

f['mp'] = 0.5*(mp + mps_and_neighbors)

# Plot the zero-population nodes
size, dpi = (14,7), 300
fig, ax = plt.subplots(figsize=size, dpi=dpi)
f.plot(column='mp', ax=ax, cmap='Spectral')
plt.axis("off")

# Legend: Red: Polygon, Blue: Multipolygon, Yellow: Neighboring precinct selected to be merged 

In [ ]:
merge_mp2 = merge_init * merge_mp
dissolve_mp2 = merge_mp2.get_dissolve()

In [ ]:
# Perform the merge
gdf_mp = perform_dissolve_gdf(gdf_init, dissolve_mp, columns_to_keep, column_labels)
graph_mp = perform_dissolve_graph(graph_init, dissolve_mp, attributes_to_sum, attributes_to_join)

In [ ]:
# We decreased by
len(graph_mp) - len(graph_init)

In [ ]:
# Plot the results

f = gdf_mp.copy()
f['mp'] = np.array([1 if i in dissolve_mp[multipolygons] else 0 for i in range(len(f))])

# Plot the multipolygons
size, dpi =(14,7), 300
fig, ax = plt.subplots(figsize=size, dpi=dpi)
f.plot(column='mp', ax=ax, cmap='Spectral')
plt.axis("off")

# Legend: Red: Polygon, Blue: Merged Multipolygon

# Section 2: One Neighbor Precincts and Separating Precincts

One essential motivation for merging precincts is to streamline the process of using Monte Carlo Markov Chain models via the flip proposal. The flip proposal is the transition between states in our Markov chain. It selects a precinct on one of the cut edges of the graph, and then changes its district assignment to the adjacent district. This transition forms a reversible Markov Chain. By simulating thousands of these flips, we hope to explore the sample space of possible districting plans. However, the flip proposal has some limitations. In particular, if precint A is contained inside precinct B (and thus has no other neighboring precincts) and precinct B is selected to be flipped, when the new districting assignment is generated, B's old district will be left discontiguous and thus invalid, because A will be a disconnected precinct in the old district. The algorithm will reject the new proposed plan due to its invalidity. Therefore, the precinct B (and precinct A) can never be flipped into a different district, and the Markov chain will not explore the whole sample space, and may be confined to a particular region of the sample space. Of course, the solution is that A and B can be flipped if they are flipped together. The simplest way to proceed is to use the normal flipping algorithm, but after having merged precincts A and B.

In [ ]:
# Choose the graph we operate on 
graph, gdf = graph_mp, gdf_mp

In [ ]:
# Find all the separating precincts, approx 5 min
ids2, neighbor_ids2, separators, merge_sp = get_separators(graph)

In [ ]:
# Confirm that donut precincts are in the same senate and house districts as the precincts they contain
for part in merge_sp.merges:
    part = list(part)
    cong, sen, house = utah.nodes[part[0]]['US_Distric'], utah.nodes[part[0]]['UT_SEN'], utah.nodes[part[0]]['UT_HOUSE']
    
    for precinct in part:
        cong1, sen1, house1 = utah.nodes[precinct]['US_Distric'], utah.nodes[part[0]]['UT_SEN'], utah.nodes[precinct]['UT_HOUSE']
    
        if (cong1 != cong) or (sen1 != sen) or (house1 != house):
            print(precinct)
        

In [ ]:
# Mark the one-neighbor nodes
f = gdf.copy()
f["one neighbor"] = np.array([1 if (i in ids2 or i in neighbor_ids2) else 0 for i in range(len(f))])

# Plot the one neighbor precincts with their one neighbor
size, dpi =(3,2), 300
fig, ax = plt.subplots(figsize=size, dpi=dpi)
f.plot(column='one neighbor', ax=ax, cmap='Spectral')
plt.axis("off")

# Legend: Red: Normal Precinct, Blue: Separating precinct, or precinct contained in it

# plt.savefig("isolated_precincts2.png", dpi=dpi, bbox_inches="tight")

# Look at Sanpete County!

In [ ]:
dissolve_sp = merge_sp.get_dissolve()

# Merge so that the new merged precincts are flippable
gdf_mp_sp = perform_dissolve_gdf(gdf_mp, dissolve_sp, columns_to_keep, column_labels)
graph_mp_sp = perform_dissolve_graph(graph_mp, dissolve_sp, attributes_to_sum, attributes_to_join)

In [ ]:
merge_mp_sp = merge_mp2 * merge_sp
dissolve_mp_sp = merge_mp_sp.get_dissolve()

In [ ]:
# Mark the merged precincts
f = gdf_mp_sp.copy()
f["one neighbor"] = np.array([1 if (n in dissolve_sp[ids2] or n in dissolve_sp[neighbor_ids2]) else 0 for n in range(len(f))])

# Plot the one neighbor precincts with their one neighbor
size, dpi =(3,2), 300
fig, ax = plt.subplots(figsize=size, dpi=dpi)
f.plot(column='one neighbor', ax=ax, cmap='Spectral')
plt.axis("off")

# Legend: Red: Normal Precinct, Blue: Separating precinct, or precinct contained in it (Merged)
# plt.savefig("isolated_precincts_merged1.png", dpi=dpi, bbox_inches="tight")

In [ ]:
# We decreased by
len(graph_mp) - len(graph_mp_sp)

In [ ]:
# No multipolygons
all(type(poly) == Polygon for poly in gdf_mp_sp['geometry'])

In [ ]:
# No separating precincts (approx 5 min to run)
separate = []
for node in graph_mp_sp:
    copy = graph_mp_sp.copy()
    copy.remove_node(node)
    separate.append(nx.is_connected(copy))
all(separate)

In [ ]:
# Check that the total population stayed the same
sum(graph_mp_sp.nodes[n]['POP100'] for n in graph_mp_sp.nodes) == sum(utah.nodes[n]['POP100'] for n in utah.nodes)

In [ ]:
len(graph_mp_sp)

Now we can package these files up:

In [ ]:
gdf_mp_sp.to_file("gdf_mp_sp.shp")

In [ ]:
pickle.dump(graph_mp_sp, open("graph_mp_sp.pkl", 'wb'))

# Section 4: Check to see which cut edges are "unflippable"

# Look at the results:

In [ ]:
# Plot with isolated precincts removed
plot_graph(gdf_mp_sp, graph_mp_sp, dpi=400, size=7, save=False, savetitle='graph_after_merging1.png')